# Scores

#### Drafter (`generate`)

`n=curr_lookahead`

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tok_ids = tokenizer.encode("Hello, my name is", return_tensors="pt")
print(tok_ids)
print(tok_ids.shape)

/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[15496,    11,   616,  1438,   318]])
torch.Size([1, 5])


/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_name)
outputs = model.generate(
    input_ids=tok_ids,
    attention_mask=torch.ones_like(tok_ids),
    max_new_tokens=10,
    do_sample=False,
    use_cache=False,
    return_dict_in_generate=True,
    output_logits=False,
    output_scores=True,
    output_hidden_states=False,
    output_attentions=False,
)
print(outputs)
# print(outputs.shape)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GenerateDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  1438,   318,  1757,    13,   314,  1101,   257,
          6260,    11,   290,   314,  1101]]), scores=(tensor([[-68.4289, -69.0398, -73.2529,  ..., -76.9532, -77.1063, -70.0094]]), tensor([[-65.0738, -66.4413, -71.7042,  ..., -79.9795, -77.5036, -68.6040]]), tensor([[-130.1885, -129.3489, -131.3793,  ..., -141.4481, -142.7361,
         -124.0823]]), tensor([[-131.9186, -130.0697, -135.7836,  ..., -141.5105, -134.1392,
         -133.0101]]), tensor([[-112.3013, -111.0004, -114.8719,  ..., -119.3571, -119.8524,
         -112.7859]]), tensor([[-108.1326, -106.7145, -110.4225,  ..., -115.2539, -114.6338,
         -107.9903]]), tensor([[ -95.4683,  -96.0403, -103.7460,  ..., -108.3876, -107.8743,
          -99.9795]]), tensor([[-83.7534, -83.9494, -85.5162,  ..., -88.0616, -89.6183, -82.4906]]), tensor([[-110.4278, -109.9855, -113.4881,  ..., -116.3650, -116.2361,
         -110.5043]]), tensor([[-144.8037, -143.3270, -147.85

In [10]:
# print all the fields of the `outputs` object
print(outputs.keys())

odict_keys(['sequences', 'scores'])


In [11]:
print(outputs.sequences)
print(tokenizer.batch_decode(outputs.sequences))

tensor([[15496,    11,   616,  1438,   318,  1757,    13,   314,  1101,   257,
          6260,    11,   290,   314,  1101]])
["Hello, my name is John. I'm a writer, and I'm"]


In [12]:
print(type(outputs.scores))
print(outputs.scores)

<class 'tuple'>
(tensor([[-68.4289, -69.0398, -73.2529,  ..., -76.9532, -77.1063, -70.0094]]), tensor([[-65.0738, -66.4413, -71.7042,  ..., -79.9795, -77.5036, -68.6040]]), tensor([[-130.1885, -129.3489, -131.3793,  ..., -141.4481, -142.7361,
         -124.0823]]), tensor([[-131.9186, -130.0697, -135.7836,  ..., -141.5105, -134.1392,
         -133.0101]]), tensor([[-112.3013, -111.0004, -114.8719,  ..., -119.3571, -119.8524,
         -112.7859]]), tensor([[-108.1326, -106.7145, -110.4225,  ..., -115.2539, -114.6338,
         -107.9903]]), tensor([[ -95.4683,  -96.0403, -103.7460,  ..., -108.3876, -107.8743,
          -99.9795]]), tensor([[-83.7534, -83.9494, -85.5162,  ..., -88.0616, -89.6183, -82.4906]]), tensor([[-110.4278, -109.9855, -113.4881,  ..., -116.3650, -116.2361,
         -110.5043]]), tensor([[-144.8037, -143.3270, -147.8548,  ..., -154.3876, -147.2709,
         -145.4146]]))


In [13]:
print(len(outputs.scores))
print(outputs.scores[0].shape)
# Stack the scores as follows.
# Instead of the existing shape of the a score (batch=1, vocab), the first dimension is
# the sequence and the second is the vocab (num_of_scores, vocab). Since the batch size
# is always 1, we can omit the batch dimension. Note that we might get `num_of_scores=1`
torch.stack(outputs.scores, dim=0).squeeze(1).shape

10
torch.Size([1, 50257])


torch.Size([10, 50257])

In [14]:
argmax = [s.argmax() for s in outputs.scores]
print(argmax)
print(tokenizer.batch_decode(argmax))

[tensor(1757), tensor(13), tensor(314), tensor(1101), tensor(257), tensor(6260), tensor(11), tensor(290), tensor(314), tensor(1101)]
[' John', '.', ' I', "'m", ' a', ' writer', ',', ' and', ' I', "'m"]


#### Verifier (`forward`)

Needs to provide the `n` scores corresponding to 
- the last `n-1` tokens, and
- the next token

In [44]:

print(f"{tok_ids=}")
print(f"{tok_ids.shape=}")

tok_ids=tensor([[15496,    11,   616,  1438,   318]])
tok_ids.shape=torch.Size([1, 5])


In [33]:
outputs = model.forward(
    input_ids=tok_ids,
    attention_mask=torch.ones_like(tok_ids),
    use_cache=False,
    return_dict=True,
    output_hidden_states=False,
    output_attentions=False,
)
outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -35.2363,  -35.3266,  -38.9754,  ...,  -44.4645,  -43.9975,
           -36.4580],
         [-112.6171, -114.5832, -116.5725,  ..., -119.0128, -118.8059,
          -111.6917],
         [ -88.7435,  -89.8644,  -93.1977,  ...,  -92.3839,  -96.1782,
           -92.1273],
         [ -51.9342,  -53.6023,  -59.3668,  ...,  -62.4015,  -66.0021,
           -57.7503],
         [ -68.4289,  -69.0398,  -73.2529,  ...,  -76.9532,  -77.1063,
           -70.0094]]], grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [40]:
outputs.logits.shape

torch.Size([1, 5, 50257])

In [46]:
logits_argmax = outputs.logits.argmax(dim=-1)
logits_argmax

tensor([[  11,  314, 1438,  318, 1757]])

In [71]:
# Add the first token at the beginning of the logits_argmax
torch.cat((tok_ids[0, :1], logits_argmax[0, :])).unsqueeze(0)

tensor([[15496,    11,   314,  1438,   318,  1757]])

`model.forward(..., return_dict=True).logits` is a tensor of shape (batch_size, seq_len, vocab_size)

In [35]:
print(f"{type(outputs.logits)=}")
print(f"{len(outputs.logits)=}")
print(f"{type(outputs.logits[0])=}")
print(f"{len(outputs.logits[0])=}")
print(f"{len(outputs.logits[-1])=}")
print(f"{outputs.logits[0][0].shape=}")

type(outputs.logits)=<class 'torch.Tensor'>
len(outputs.logits)=1
type(outputs.logits[0])=<class 'torch.Tensor'>
len(outputs.logits[0])=5
len(outputs.logits[-1])=5
outputs.logits[0][0].shape=torch.Size([50257])


# Preemption

Proof-of-concept that we can preempt a long-running call to `transformers`.

# Rejection sampling

In [76]:
torch.tensor([ True, False,  True,  True,  True, False]).nonzero()[0]

tensor([0])

# Temperature & sampling

In [12]:
import accelerate
from poc.actual.utils import set_hf_cache
from transformers import AutoModelForCausalLM
import torch

tok_ids = torch.tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
            13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
           382,  14711,  30151,    512,   3923,   1053,    387,    279,   1888,
           955,    315,  10368,    369,    264,   1732,    889,    706,  55652,
          1980,  14711,   6075,    512,   2520,    264,   1732,    449,  55652,
            11,    279,   1888,    955,    315,  10368,    374,   3629,   3428,
            12,  58400,   7640,    430,   1541,    956,   2231,  27639,  26800   ]])

cache_dir = set_hf_cache()
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", device_map="auto", load_in_8bit=True, cache_dir=cache_dir)

model.generate(
    input_ids=tok_ids,
    attention_mask=torch.ones_like(tok_ids),
    max_new_tokens=10,
    do_sample=False,
    use_cache=False,
    return_dict_in_generate=True,
    output_scores=True,
    output_logits=False,
    output_hidden_states=False,
    output_attentions=False,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Main: Set Hugging Face cache directory to /workspace/hf_cache
Main: Set Hugging Face home directory to /workspace/hf_cache


Loading checkpoint shards: 100%|██████████| 4/4 [01:25<00:00, 21.46s/it]
/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-

GenerateDecoderOnlyOutput(sequences=tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
             13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
            382,  14711,  30151,    512,   3923,   1053,    387,    279,   1888,
            955,    315,  10368,    369,    264,   1732,    889,    706,  55652,
           1980,  14711,   6075,    512,   2520,    264,   1732,    449,  55652,
             11,    279,   1888,    955,    315,  10368,    374,   3629,   3428,
             12,  58400,   7640,    430,   1541,    956,   2231,  27639,  26800,
            389,    279,  35358,     13,  26379,   2997,  24269,     11,  33162,
             11]]), scores=(tensor([[ 7.3828,  9.3125,  3.7773,  ..., -2.1758, -2.1758, -2.1758]]), tensor([[ 5.9648,  6.2812,  1.8799,  ..., -1.7920, -1.7910, -1.7920]]), tensor([[ 5.5781,  5.1875,  3.1113,  ..., -3.6992, -3.6992, -3.6992]]), tensor([[10.2734,  7.1719,  1.8867,  ..., -1.7637, -1.7637, -1.7637]]), 

In [13]:
all_seqs = []
for _ in range(10):
    outputs = model.generate(
        input_ids=tok_ids,
        attention_mask=torch.ones_like(tok_ids),
        max_new_tokens=10,
        do_sample=False,
        use_cache=False,
        return_dict_in_generate=True,
        output_scores=True,
        output_logits=False,
        output_hidden_states=False,
        output_attentions=False,
    )
    all_seqs.append(outputs.sequences)
all_seqs

/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/root/.cache/pypoetry/virtualenvs/distributed-speculative-inference-w7-NjH3e-py3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1885: UserWarning: You are call

[tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
              13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
             382,  14711,  30151,    512,   3923,   1053,    387,    279,   1888,
             955,    315,  10368,    369,    264,   1732,    889,    706,  55652,
            1980,  14711,   6075,    512,   2520,    264,   1732,    449,  55652,
              11,    279,   1888,    955,    315,  10368,    374,   3629,   3428,
              12,  58400,   7640,    430,   1541,    956,   2231,  27639,  26800,
             389,    279,  35358,     13,  26379,   2997,  24269,     11,  33162,
              11]]),
 tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
              13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
             382,  14711,  30151,    512,   3923,   1053,    387,    279,   1888,
             955,    315,  10368,    369,    264,   1732,    889,    706,  55

In [14]:
[(all_seqs[0] == s).all() for s in all_seqs]

[tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True)]

In [ ]:
model(tok_ids)